In [1]:
import torch
from transformers import pipeline

pipe = pipeline("text-generation", model="TinyLlama/TinyLlama-1.1B-Chat-v1.0", torch_dtype=torch.bfloat16, device_map="auto")



config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

In [17]:
import bs4
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.indexes import VectorstoreIndexCreator
from sentence_transformers import SentenceTransformer
from langchain.embeddings import OpenAIEmbeddings
from langchain.embeddings import HuggingFaceBgeEmbeddings

##Load and split documents
bs4_strainer = bs4.SoupStrainer(class_=("main-content"))
loader = WebBaseLoader(
    web_paths=("https://deltek.com/en","https://www.deltek.com/en/about/contact-us", "https://www.deltek.com/en/small-business", "https://www.deltek.com/en/customers",
               "https://www.deltek.com/en/support", "https://www.deltek.com/en/partners"),
    bs_kwargs={"parse_only": bs4_strainer},
)
docs = loader.load()



text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)
all_splits_text = [split.page_content for split in all_splits]







##open source embedding model
model_name = "BAAI/bge-large-en-v1.5"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity

embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
    # query_instruction="What is Deltek?"
)

query = "what does Deltek do?"
db = FAISS.from_documents(all_splits, embeddings)

##get relevant docs from vectorstore
relevant_docs = db.similarity_search(query)
formatted_docs = '\n'.join(doc.page_content for doc in relevant_docs)
print(formatted_docs)


Register Now to Save















Deltek Powers Projects
Deltek delivers software and information solutions that enable superior levels of project intelligence, management and collaboration. Our industry-focused expertise makes your projects successful and helps you achieve performance that maximizes productivity at every stage of the project lifecycle that fuels your business.

Learn More About our Vision





The 5 Phases of the Project Lifecycle








                    Win
                







                    Manage
                







                    Develop
                







                    Deliver
                







                    Measure
                










Win


Win
and discover new opportunities by streamlining your business development.







Manage


Manage
your projects efficiently and cost effectively while optimizing resources.







Develop
Technology innovation is essential for small businesses seeking growth and a compe

In [18]:
# We use the tokenizer's chat template to format each message - see https://huggingface.co/docs/transformers/main/en/chat_templating
messages = [
    {
        "role": "system",
        "content": "You are a friendly chatbot who responds about Deltek"
    },
    {"role": "user", "content": "context: "+ formatted_docs + "query: " + query},
]
prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
start_out = len(prompt)
print(outputs[0]["generated_text"][start_out:])

The Deltek Partner Network is Deltek's technology partners who are essential for deploying new capabilities, integrating third-party solutions, and achieving greater results in an accelerated timeframe. Deltek's partners are selected based on their experience, expertise, and commitment to delivering innovative software solutions that meet the unique needs of small businesses. They are skilled in leveraging Deltek's industry-specific software to optimize performance, reduce costs, and maximize productivity, providing small businesses with the security, privacy, and flexibility they need. Partners can access a variety of resources, including product training, implementation support, and marketing and sales tools, to help them successfully implement Deltek's solutions and achieve their business goals.
